In [1]:
import cplex
from cplex.exceptions import CplexError
from docplex.mp.model import *
import random
import matplotlib.pyplot as plt
import docplex.mp.model as Solution
import docplex.cp.parameters as param
import numpy as np
import pandas as pd
import math as math
import csv

In [2]:
rnd = np.random
rnd.seed(0)

In [3]:
def maximum(a, b):
    if a >= b:
        return a
    else:
        return b

In [4]:
data = []
cost = []
t = [] # thời gian di chuyển từ điểm i đến điểm j

# Read the CSV file
with open('distance_matrix.csv', 'r', encoding="utf-8-sig") as file:
    csv_reader = csv.reader(file)

    # Iterate over each row in the CSV file
    for row in csv_reader:
        # Convert the elements in the row from strings to integers
        distance = [int(float(element)) for element in row]
        data.append(distance)
        time_trans = [int(float(element)) * 6 for element in row]
        t.append(time_trans)
    

Lấy các ràng buộc từ khách hàng

In [5]:
data_demands = []
ready_time = []
due_time = []
service_time = []

# Read the customer data from the text file
with open('input.txt', 'r') as file:
    # Skip the header line
    next(file)

    # Process each line in the file
    for line in file:
        # Remove leading/trailing whitespace and split the line into attributes
        attributes = line.strip().split()
        if len(attributes) >= 7:
        # Convert customer number, X coordinate, and Y coordinate to integers
            demand = int(attributes[3])
            redtime = int(attributes[4])
            dutime  =  int(attributes[5])
            sertime = int(attributes[6])

            # Add the dictionary to the data list
            data_demands.append(demand)
            ready_time.append(redtime)
            due_time.append(dutime)
            service_time.append(sertime)

Khai báo các chỉ số cơ bản

In [6]:
# n  = len(data) - 1 #số lượng khách hàng
n = 100
Q = 1500 #sức chứa của xe
K = 3 #số lượng xe
V = [i for i in range(1,K+1)] # set of vehicle
C = [i for i in range(1,n+1)] #set of client
N = [0] + C #set of node (warehouse + client)
q = {i:data_demands[i] for i in C} #yêu cầu của khách hàng i
s = {i:service_time[i] for i in C} #thời gian phục vụ khách hàng i
a = {i:ready_time[i] for i in C} #thời gian sớm nhất phục vụ khách hàng i
b = {i:due_time[i] for i in C} #thời gian muộn nhất phục vụ khách hàng i
sum(q.values())

1810

In [7]:
A = [(i, j) for i in N for j in N if i != j]
c = {(i, j): data[i][j] for i, j in A} #chi phí di chuyển từ điểm i đến điểm j

In [8]:
mdl = Model('VRPTW')

In [9]:
x = mdl.binary_var_dict(A, name='x')
w = mdl.continuous_var_dict(C, name='w') #thời gian bắt đầu phục vụ 
u = mdl.continuous_var_dict(C, ub=Q, name='u') #số lượng hàng trên phương tiện


In [10]:
mdl.minimize(mdl.sum(c[i, j]*x[i,j] for i, j in A))
mdl.add_constraints(mdl.sum(x[i, j] for j in N if j != i) == 1 for i in C )
mdl.add_constraints(mdl.sum(x[i, j] for i in N if i != j) == 1 for j in C )
mdl.add_indicator_constraints(mdl.indicator_constraint(x[i, j], w[i] + s[i] + t[i][j] <= w[j]) for i, j in A if i != 0 and j != 0)
mdl.add_constraints(w[i] <= b[i] for i in C)
mdl.add_constraints(w[i] >= a[i] for i in C)
mdl.add_indicator_constraints(mdl.indicator_constraint(x[i, j], u[i]+q[j] == u[j]) for i, j in A if i != 0 and j != 0)
mdl.add_constraints(u[i] >= q[i] for i in C)
solution = mdl.solve(log_output=True)

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 5702.000000 after 0.00 sec. (0.91 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 313 rows and 7221 columns.
Aggregator did 15 substitutions.
Reduced MIP has 4508 rows, 6578 columns, and 17727 nonzeros.
Reduced MIP has 2864 binaries, 0 generals, 0 SOSs, and 3447 indicators.
Presolve time = 0.09 sec. (56.96 ticks)
Probing time = 0.11 sec. (16.69 ticks)
Tried aggregator 2 times.
Detecting symmetries...
Aggregator did 67 substitutions.
Reduced MIP has 4441 rows, 6511 columns, and 17593 nonzeros.
Reduced MIP has 2864 binaries, 0 generals, 0 SOSs, and 3447 indicators.
Presolve time = 0.05 sec. (17.76 ticks)
Probing time = 0.03 sec. (4.74 ticks)
Clique table members: 887.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 12 threads.
Root relaxation solution time = 0.03 sec. (10.9

In [11]:
solution.display()

solution for: VRPTW
objective: 1339
status: OPTIMAL_SOLUTION(2)
x_0_5 = 1
x_0_13 = 1
x_0_17 = 1
x_0_20 = 1
x_0_24 = 1
x_0_31 = 1
x_0_32 = 1
x_0_42 = 1
x_0_43 = 1
x_0_57 = 1
x_0_65 = 1
x_0_67 = 1
x_0_76 = 1
x_0_81 = 1
x_0_90 = 1
x_0_96 = 1
x_0_98 = 1
x_1_0 = 1
x_2_1 = 1
x_3_7 = 1
x_4_0 = 1
x_5_3 = 1
x_6_4 = 1
x_7_8 = 1
x_8_97 = 1
x_9_6 = 1
x_10_11 = 1
x_11_9 = 1
x_12_0 = 1
x_13_15 = 1
x_14_12 = 1
x_15_16 = 1
x_16_14 = 1
x_17_18 = 1
x_18_19 = 1
x_19_28 = 1
x_20_10 = 1
x_21_0 = 1
x_22_21 = 1
x_23_0 = 1
x_24_25 = 1
x_25_27 = 1
x_26_23 = 1
x_27_29 = 1
x_28_22 = 1
x_29_30 = 1
x_30_26 = 1
x_31_35 = 1
x_32_33 = 1
x_33_37 = 1
x_34_0 = 1
x_35_38 = 1
x_36_34 = 1
x_37_39 = 1
x_38_51 = 1
x_39_36 = 1
x_40_44 = 1
x_41_40 = 1
x_42_53 = 1
x_43_41 = 1
x_44_46 = 1
x_45_48 = 1
x_46_45 = 1
x_47_0 = 1
x_48_47 = 1
x_49_0 = 1
x_50_52 = 1
x_51_50 = 1
x_52_49 = 1
x_53_58 = 1
x_54_56 = 1
x_55_54 = 1
x_56_64 = 1
x_57_55 = 1
x_58_60 = 1
x_59_0 = 1
x_60_59 = 1
x_61_66 = 1
x_62_74 = 1
x_63_0 = 1
x_64_68 = 1
x_65_63 

In [12]:
active_arcs = [a for a in A if x[a].solution_value == 1]
active_arcs

[(0, 5),
 (0, 13),
 (0, 17),
 (0, 20),
 (0, 24),
 (0, 31),
 (0, 32),
 (0, 42),
 (0, 43),
 (0, 57),
 (0, 65),
 (0, 67),
 (0, 76),
 (0, 81),
 (0, 90),
 (0, 96),
 (0, 98),
 (1, 0),
 (2, 1),
 (3, 7),
 (4, 0),
 (5, 3),
 (6, 4),
 (7, 8),
 (8, 97),
 (9, 6),
 (10, 11),
 (11, 9),
 (12, 0),
 (13, 15),
 (14, 12),
 (15, 16),
 (16, 14),
 (17, 18),
 (18, 19),
 (19, 28),
 (20, 10),
 (21, 0),
 (22, 21),
 (23, 0),
 (24, 25),
 (25, 27),
 (26, 23),
 (27, 29),
 (28, 22),
 (29, 30),
 (30, 26),
 (31, 35),
 (32, 33),
 (33, 37),
 (34, 0),
 (35, 38),
 (36, 34),
 (37, 39),
 (38, 51),
 (39, 36),
 (40, 44),
 (41, 40),
 (42, 53),
 (43, 41),
 (44, 46),
 (45, 48),
 (46, 45),
 (47, 0),
 (48, 47),
 (49, 0),
 (50, 52),
 (51, 50),
 (52, 49),
 (53, 58),
 (54, 56),
 (55, 54),
 (56, 64),
 (57, 55),
 (58, 60),
 (59, 0),
 (60, 59),
 (61, 66),
 (62, 74),
 (63, 0),
 (64, 68),
 (65, 63),
 (66, 0),
 (67, 62),
 (68, 69),
 (69, 0),
 (70, 73),
 (71, 70),
 (72, 61),
 (73, 80),
 (74, 72),
 (75, 0),
 (76, 71),
 (77, 79),
 (78, 77),
 (

In [13]:
def find_all_routes(data_list):
    routes = []
    pre_value = -1
    current_route = []
    while True:
        if len(data_list) == 0:
            break
        for i in data_list:
            if pre_value == 0:
                routes.append(current_route.copy())
                current_route.clear()
                pre_value = -1
                break
            if i[0] == 0 and len(current_route) == 0:
                current_route.extend([i[0], i[1]])
                pre_value = i[1]
                data_list.remove(i)
                continue
            elif i[0] == pre_value:
                current_route.append(i[1])
                pre_value = i[1]
                data_list.remove(i)
                if len(data_list) == 0:
                    routes.append(current_route)
                    break
    return routes

In [14]:
def format_route(route):
    return " -> ".join(str(node) for node in route)

In [15]:
result_routes = find_all_routes(active_arcs)
if result_routes == -1:
    print("No result")
else:
    print("solution for VRPTW")
    print(f"objective: {solution.get_objective_value()}")
    for i, route in enumerate(result_routes, 1):
        print(f"Route {i}: {format_route(route)}")


solution for VRPTW
objective: 1339.0
Route 1: 0 -> 5 -> 3 -> 7 -> 8 -> 97 -> 100 -> 99 -> 75 -> 0
Route 2: 0 -> 13 -> 15 -> 16 -> 14 -> 12 -> 0
Route 3: 0 -> 17 -> 18 -> 19 -> 28 -> 22 -> 21 -> 0
Route 4: 0 -> 20 -> 10 -> 11 -> 9 -> 6 -> 4 -> 0
Route 5: 0 -> 24 -> 25 -> 27 -> 29 -> 30 -> 26 -> 23 -> 0
Route 6: 0 -> 31 -> 35 -> 38 -> 51 -> 50 -> 52 -> 49 -> 0
Route 7: 0 -> 32 -> 33 -> 37 -> 39 -> 36 -> 34 -> 0
Route 8: 0 -> 42 -> 53 -> 58 -> 60 -> 59 -> 0
Route 9: 0 -> 43 -> 41 -> 40 -> 44 -> 46 -> 45 -> 48 -> 47 -> 0
Route 10: 0 -> 57 -> 55 -> 54 -> 56 -> 64 -> 68 -> 69 -> 0
Route 11: 0 -> 65 -> 63 -> 0
Route 12: 0 -> 67 -> 62 -> 74 -> 72 -> 61 -> 66 -> 0
Route 13: 0 -> 76 -> 71 -> 70 -> 73 -> 80 -> 0
Route 14: 0 -> 81 -> 78 -> 77 -> 79 -> 0
Route 15: 0 -> 90 -> 87 -> 86 -> 89 -> 0
Route 16: 0 -> 96 -> 95 -> 94 -> 92 -> 93 -> 2 -> 1 -> 0
Route 17: 0 -> 98 -> 83 -> 82 -> 84 -> 85 -> 88 -> 91 -> 0
